In [1]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
df_industries = pd.read_csv('D:/Study-3rd/DSP/OP-heist-project/data/Indices/industries_indexes.csv',encoding = 'iso-8859-1')
df_regions = pd.read_csv('D:/Study-3rd/DSP/OP-heist-project/data/Indices/regional_indexes.csv',encoding = 'iso-8859-1')
df_regions

,Unnamed: 0,Time,Area,Dominant Industry,Regional export,Regional import,Regional trade dependency,Regional import-Export imbalance,Regional export Growth,Regional import Growth,Export concentration
0,0,2015,MK01,G,1.691202e+10,3.347404e+10,61.084771,0.328702,0.000000,0.000000,13.346160
1,1,2015,MK02,Q,4.646912e+09,3.430935e+09,49.373111,-0.150532,0.000000,0.000000,16.991052
2,2,2015,MK04,C,3.348066e+09,1.207024e+09,55.997258,-0.470033,0.000000,0.000000,19.388457
3,3,2015,MK05,C,1.288965e+09,7.219806e+08,36.319109,-0.281949,0.000000,0.000000,16.681200
4,4,2015,MK06,C,4.161019e+09,3.159014e+09,41.505377,-0.136885,0.000000,0.000000,17.276955
...,...,...,...,...,...,...,...,...,...,...,...
109,109,2020,MK16,Q,1.892938e+09,2.028934e+09,148.775192,0.034676,0.103754,0.125935,19.992198
110,110,2020,MK17,Q,1.620991e+09,1.856990e+09,22.960239,0.067855,-0.289662,-0.152060,18.946149
111,111,2020,MK18,Q,3.950119e+08,1.210036e+08,20.525997,-0.531008,-0.102246,0.234731,18.777516
112,112,2020,MK19,Q,3.445914e+09,1.600960e+09,73.038080,-0.365564,-0.097478,-0.187762,19.179797


In [3]:
# List of all regions
regions = df_regions['Area'].unique().tolist()

#Years for future predictions
future_years = [2021, 2022, 2023]

all_export_growth_preds = []
all_trade_dependency_preds = []
all_import_export_imbalance_preds = []
all_import_growth_preds = []
all_export = []
all_import = []
all_import_export = []
all_trade_dependency = []
exports = []
imports = []
imbalances = []
trade_dependencies = []
for region_idx, region in enumerate(regions):
    condition = df_regions['Area'] == region
    filtered_df = df_regions[condition]
    if filtered_df.shape[0] == 0:
        continue  
    #Features
    X_export_growth = filtered_df['Time'].values.reshape(-1, 1)
    y_export_growth = filtered_df['Regional export Growth'].values
    X_trade_dependency = filtered_df['Time'].values.reshape(-1, 1)
    y_trade_dependency = filtered_df['Regional trade dependency'].values
    X_import_export_imbalance = filtered_df['Time'].values.reshape(-1, 1)
    y_import_export_imbalance = filtered_df['Regional import-Export imbalance'].values
    X_import_growth = filtered_df['Time'].values.reshape(-1, 1)
    y_import_growth = filtered_df['Regional import Growth'].values
    all_export = y_export_growth
    all_import = y_import_growth
    all_imbalance = y_import_export_imbalance
    all_trade_dependency = y_trade_dependency
    # Polynomial regression
    degree = 1  # Polynomial degrees
    poly = PolynomialFeatures(degree=degree)
    X_export_growth_poly = poly.fit_transform(X_export_growth)
    X_trade_dependency_poly = poly.fit_transform(X_trade_dependency)
    X_import_export_imbalance_poly = poly.fit_transform(X_import_export_imbalance)
    X_import_growth_poly = poly.fit_transform(X_import_growth)
    # Declare variables for saving
    export_growth_preds = []
    trade_dependency_preds = []
    import_export_imbalance_preds = []
    import_growth_preds = []
    #  Models
    export_growth_model = LinearRegression()
    trade_dependency_model = LinearRegression()
    import_export_imbalance_model = LinearRegression()
    import_growth_model = LinearRegression()
    # Fitting
    export_growth_model.fit(X_export_growth_poly, y_export_growth)
    trade_dependency_model.fit(X_trade_dependency_poly, y_trade_dependency)
    import_export_imbalance_model.fit(X_import_export_imbalance_poly, y_import_export_imbalance)
    import_growth_model.fit(X_import_growth_poly, y_import_growth)
    # Save results
    future_preds_export_growth = []
    future_preds_trade_dependency = []
    future_preds_import_export_imbalance = []
    future_preds_import_growth = []
    for year in future_years:
        X_future = np.array([[year]])
        X_future_poly = poly.transform(X_future)
        export_growth_pred = export_growth_model.predict(X_future_poly)
        trade_dependency_pred = trade_dependency_model.predict(X_future_poly)
        import_export_imbalance_pred = import_export_imbalance_model.predict(X_future_poly)
        import_growth_pred = import_growth_model.predict(X_future_poly)
        future_preds_export_growth.append(export_growth_pred)
        future_preds_trade_dependency.append(trade_dependency_pred)
        future_preds_import_export_imbalance.append(import_export_imbalance_pred)
        future_preds_import_growth.append(import_growth_pred)
        all_export = np.append(all_export,export_growth_pred)
        all_import = np.append(all_import,import_growth_pred)
        all_imbalance = np.append(all_imbalance,import_export_imbalance_pred)
        all_trade_dependency = np.append(all_trade_dependency,trade_dependency_pred)
    # List to save the predictions
    all_export_growth_preds.append(future_preds_export_growth)
    all_trade_dependency_preds.append(future_preds_trade_dependency)
    all_import_export_imbalance_preds.append(future_preds_import_export_imbalance)
    all_import_growth_preds.append(future_preds_import_growth)
    exports.append(all_export)
    imports.append(all_import)
    imbalances.append(all_imbalance)
    trade_dependencies.append(all_trade_dependency)
    

In [4]:
time = [2015,2016,2017,2018,2019,2020,2021,2022,2023]
df_exports = pd.DataFrame()
df_exports['Time'] = time
df_imports = pd.DataFrame()
df_imports['Time'] = time
df_imbalances = pd.DataFrame()
df_imbalances['Time'] = time
df_trade_dependencies = pd.DataFrame()
df_trade_dependencies['Time'] = time

for export in exports:
    for region in regions:
        df_exports[region] = export
for imp in imports:
    for region in regions:
        df_imports[region] = imp
for imbalance in imbalances:
    for region in regions:
        df_imbalances[region] = imbalance
for trade_dependency in trade_dependencies:
    for region in regions:
        df_trade_dependencies[region] = trade_dependency       

In [5]:
df_exports.to_csv('D:/Study-3rd/DSP/OP-heist-project/data/forecast_values/exports.csv')
df_imports.to_csv('D:/Study-3rd/DSP/OP-heist-project/data/forecast_values/imports.csv')
df_imbalances.to_csv('D:/Study-3rd/DSP/OP-heist-project/data/forecast_values/imbalances.csv')
df_trade_dependencies.to_csv('D:/Study-3rd/DSP/OP-heist-project/data/forecast_values/trade_dependencies.csv')